I am creating the test sets at trace level that can be used after the models have been trained on the entire graph. It required more than 12 hours. At the end 4234 traces were obtained.

In [1]:
import pandas as pd
import numpy as np
import os
import joblib
import zipfile
from tqdm import tqdm
import contextlib

seed = 1234

In [2]:

# # specify the name of the zip file and the directory to extract the contents
# zip_file_name = f'MSCallGraph_joined_reduced.zip'
# extract_dir = ""

# # create a ZipFile object and extract the contents to the specified directory
# with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)


In [3]:
complete_df = pd.read_csv('MSCallGraph_joined_io.csv')
complete_df

,traceid,um,rpctype,dm
0,0b133c1915919238193454000e5d37,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc_out,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...
1,0b133c1915919238193454000e5d37,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc_out,fd6d86bd0fd550e717c1fdb82a33190a9fef216d87d535...
2,0b133c1915919238193454000e5d37,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,userDefined_out,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
3,0b133c1915919238193454000e5d37,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc_in,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
4,0b133c1915919238193454000e5d37,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc_out,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
...,...,...,...,...
222509712,0b14339415919667189434000e499f,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,rpc_out,2abd05990aa9eb81a3eaa333829ec355eaee57fbecc9e8...
222509713,0b14339415919667189434000e499f,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,rpc_in,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...
222509714,0b14339415919667189434000e499f,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,rpc_in,643bce6326e327bb36361c31d1ad0cf9ee8d5767175482...
222509715,0b14339415919667189434000e499f,643bce6326e327bb36361c31d1ad0cf9ee8d5767175482...,rpc_out,e9916393b398199ef017f67a1904dd9c30323405c6f77c...


In [4]:
# df = triplets_df
df = complete_df.drop(['traceid'],axis=1)
df = df.drop_duplicates()
df

,um,rpctype,dm
0,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc_out,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...
1,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc_out,fd6d86bd0fd550e717c1fdb82a33190a9fef216d87d535...
2,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,userDefined_out,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc_in,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
4,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc_out,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
...,...,...,...
222400013,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,rpc_in,1224ab4a5244545b7333a290aaccc95e49f2a793457d21...
222400014,1224ab4a5244545b7333a290aaccc95e49f2a793457d21...,db_out,1ac258b5230c8d9dfb28dcad1ced301a9b475b58b37025...
222447693,cc89071de722ad78455c531ebeefab3380ed933ffcd0c8...,rpc_in,159eafe7e8d59656f3f1b913cca6b8fb0127e62a7019f5...
222467609,5d887bdb9fd24cea365fb7ce018526df1d39b11630081b...,db_out,8ee7c139f721a23915ca012d2518eddbdc7ad1ced8b3da...


In [5]:
entities = set(df['dm']) | set(df['um'])
len(entities)

16444

In [6]:
baseline = []
for entity in entities:
    baseline.append(df[df['dm'] == entity].head(1))
    baseline.append(df[df['um'] == entity].head(1))

In [7]:
baseline_df = pd.concat(baseline)
baseline_df = baseline_df.drop_duplicates()
baseline_df

,um,rpctype,dm
41869,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,db_out,9d54b689f3228646e4d341dd7bfa1c88b405ffd7051902...
16784905,9cd44f1b3a60567136e9582fe05e3cef48af5afe220acc...,db_out,a1f8e77131ec91237271155d16b214f4ae56a59dd84576...
12487,3d6673105b7850d8b917bad14ffbace2333a6f34f6395f...,db_out,ae715aa895509e0cd15df2b2f6964e8b783b3b6729a1d2...
2150079,daa312b7d43ebc40418dac11463874b27bd7d320ca79fd...,db_out,d44017fe2971acde82d91e34f762b925a79b301cbdc601...
15872206,384c1fd74efa0dce3dc948ea67b088115573f9f76792be...,db_out,9063beb6ec1dbe50f082031b8cd692503280682ab2669c...
...,...,...,...
42275986,f06b741d0362b424f7ebd307dd710252253d609b1197aa...,db_out,9d6b6445eea0ab93f40c85489550b048ccbd7eab757977...
212625,edd5edf6fabedb37dc011f2ff1dea0a07350815be8361d...,db_out,dd87cae238217e1c404e9dceb077c5e6b12852432b6096...
40767380,48848e19eab5d282a78cdfa69d64b15dd6a831130a7a43...,db_out,d27455ddd9964752710a0a30621dc7f5ccf4505f1dda2d...
812267,47e0ecaf3f7bc79aa518952e989c45ca9f9bfe103dfeb9...,rpc_out,a098c22586f6fe92476fb97361ee036a31d2c5da7db1b6...


In [8]:
n_train_items = int(len(df)*0.64)
n_items_to_add = n_train_items - len(baseline_df)
print("the number of items to add to the train set is: ",n_items_to_add)

the number of items to add to the train set is:  22963


In [9]:
not_baseline_df = df.merge(baseline_df, how='outer', indicator=True).loc[lambda x : x['_merge']=='left_only'].drop('_merge',axis= 1)
not_baseline_df

,um,rpctype,dm
9,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,userDefined_out,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
11,20119b0b7869f98acb0f0f7557893474a7b84c34e50ec6...,rpc_in,4ab265f54516248ee8873be7d6441912456ce17e84f399...
12,4ab265f54516248ee8873be7d6441912456ce17e84f399...,rpc_out,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
13,4ab265f54516248ee8873be7d6441912456ce17e84f399...,http_out,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
19,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc_in,e6c95bef37d936ebb375bf135ded88c96eb9257a1d3d9d...
...,...,...,...
64864,eb7ce54148779f812f2c7f07f18ef4ed269043b9f6b959...,db_out,db984ae0fed93d22fbcca65eb1ec1b25b4faa71167c580...
64865,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,rpc_in,1224ab4a5244545b7333a290aaccc95e49f2a793457d21...
64867,cc89071de722ad78455c531ebeefab3380ed933ffcd0c8...,rpc_in,159eafe7e8d59656f3f1b913cca6b8fb0127e62a7019f5...
64868,5d887bdb9fd24cea365fb7ce018526df1d39b11630081b...,db_out,8ee7c139f721a23915ca012d2518eddbdc7ad1ced8b3da...


In [10]:
not_baseline_train = not_baseline_df.sample(n_items_to_add,random_state = seed)
train_df = pd.concat([baseline_df, not_baseline_train])
train_df

,um,rpctype,dm
41869,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,db_out,9d54b689f3228646e4d341dd7bfa1c88b405ffd7051902...
16784905,9cd44f1b3a60567136e9582fe05e3cef48af5afe220acc...,db_out,a1f8e77131ec91237271155d16b214f4ae56a59dd84576...
12487,3d6673105b7850d8b917bad14ffbace2333a6f34f6395f...,db_out,ae715aa895509e0cd15df2b2f6964e8b783b3b6729a1d2...
2150079,daa312b7d43ebc40418dac11463874b27bd7d320ca79fd...,db_out,d44017fe2971acde82d91e34f762b925a79b301cbdc601...
15872206,384c1fd74efa0dce3dc948ea67b088115573f9f76792be...,db_out,9063beb6ec1dbe50f082031b8cd692503280682ab2669c...
...,...,...,...
58389,35c56101f54b77bf06b6ba688e09f401e13c906576fd7b...,db_out,54051d5ad7a223e239c7f993b5d414a31ff16ffec22842...
42795,5f3cd749ad44311b45da979c0b14517019690e8587f6f7...,rpc_in,820d792605059d1c40fe6f34e105b99ae046197bf8f3c8...
54405,fb1dc665d2cce93dd583d61d9565a6826a8c3ac27075e9...,rpc_in,f10612cc80a4777bcb0f9025041b3e4a9d8de8b20bfd2d...
52076,379e23daa893e9f183169734b12ee6c2ca81ca05d270a8...,rpc_out,7d53863bd1aa3d9e1fe4a8b4ff37800da75b3144796adc...


In [11]:
remaining_df = not_baseline_df.drop(not_baseline_train.index) 

In [12]:
validation_df = remaining_df.sample(frac = 0.44, random_state = seed)
validation_df

,um,rpctype,dm
26941,19612380d038043e051cf922650c420311bc292cdc32c5...,rpc_out,561cfc0625d5c797528d6a9027c8198b39915b221ac2b7...
51232,bf4db78123a0031936185b2f2387d4bec7212905459eed...,db_out,04d03e7464b4ba98cc19756bcda54140553ff84a67a384...
45693,d849617ba403c352dbc8a444ee366dbca049d4a49cd49b...,rpc_in,a2f4cc0518d7224f846bcf6efcf46be5c37f2c2045bf84...
5550,8ca2865ffd758cda9104100aa8a7d8cd23357d17d54621...,rpc_in,23bdd392279c3f7f575cdcca6c022c7403272aea434e9e...
39710,2024dd56b448408239935e936c2207dd15fcdb98e4b7c6...,db_out,d9c63ef9e4abef1240824ab1a2f503c5406a4e5244b964...
...,...,...,...
7749,a9cd30c90ea19c56f4cd480ff0c31461a7934c0500c447...,rpc_in,d2c895bd5c7de55f381310fd1b94ce9d518bf4ac22e22f...
25211,dad93363c30d440df0bae043024a1367f25475c8501ede...,rpc_in,3322acdce65a8353e2f8a32a2bf0474526cfc62c3d1328...
6851,0e196e772ff517533a7396537ab970729fbbc9ae2f5685...,mc_out,2720b0db4803852457cc9fa53872f47f65f872a476e6f4...
17625,86562d64f056f9c8c58aecf349f090a30d1e5dd397fa2c...,mc_out,c964e78e1ee643babee3c48807207a39596bdc816e5ceb...


In [13]:
test_df = remaining_df.drop(validation_df.index)  # drop the sampled rows to get the second DataFram
test_df

,um,rpctype,dm
11,20119b0b7869f98acb0f0f7557893474a7b84c34e50ec6...,rpc_in,4ab265f54516248ee8873be7d6441912456ce17e84f399...
12,4ab265f54516248ee8873be7d6441912456ce17e84f399...,rpc_out,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
13,4ab265f54516248ee8873be7d6441912456ce17e84f399...,http_out,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
20,4ab265f54516248ee8873be7d6441912456ce17e84f399...,rpc_out,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
57,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc_in,b53f8e2c1722bba4f5819ecf0d6d1dc522825977126aee...
...,...,...,...
64846,bc529ee59852e87efa2e41286d68dc59c973bbf8131324...,rpc_out,99f2e7b501f50db9b4089242a5d3e1aba334c32e0c718b...
64848,c7da4297c678a12e5298bbebed6599928424f56cf4b1e1...,db_out,5ff3630fb1b1eb87e7501f7ca3a73e8cd55121a63a1d5c...
64859,0e196e772ff517533a7396537ab970729fbbc9ae2f5685...,db_out,dfc72b449a65283e6038992b83a9fd2aef4a62444f1250...
64860,8c668296f4ee5134867e07e2196acd9e2f76181bc6983a...,db_out,1ef3e9820a0cd97ef806a682fa6484d44758aaa46290ba...


In [14]:
test_traces_df = complete_df.iloc[test_df.index].drop(columns=['um','rpctype','dm'],axis=1)
test_traces_df

,traceid
11,0b1339ef15919237750706000eded3
12,0b133c1915919238193454000e5d37
13,0b133c1915919238193454000e5d37
20,0b133c1915919238193454000e5d37
57,0b11355e15919238739898000e4315
...,...
64846,0b14275f15919238843807000e17aa
64848,0b14275f15919238843807000e17aa
64859,0b1428f515919237160953000e4403
64860,0b1428f515919237160953000e4403


In [15]:
test_traces = list(set(test_traces_df['traceid']))
len(test_traces)

4234

In [16]:
def create_train_test_trace(complete_df, trace):
    trace_df = complete_df[complete_df['traceid']==trace].drop_duplicates()
    trace_triplets_df = trace_df.drop(columns=['traceid'],axis=1).drop_duplicates()

    trace_entities = set(trace_triplets_df['dm']) | set(trace_triplets_df['um'])
    trace_relations = set(trace_triplets_df['rpctype'])
    trace_baseline = []
    for entity in trace_entities:
        trace_baseline.append(trace_triplets_df[trace_triplets_df['dm'] == entity].head(1))
        trace_baseline.append(trace_triplets_df[trace_triplets_df['um'] == entity].head(1))

    for relation in trace_relations:
        trace_baseline.append(trace_triplets_df[trace_triplets_df['rpctype'] == relation].head(1))


    trace_train_df = pd.concat(trace_baseline)
    trace_train_df = trace_train_df.drop_duplicates()

    trace_test_df = trace_triplets_df.drop(trace_train_df.index) 

    return trace_train_df, trace_test_df

In [17]:
trace_train_df_list = []
trace_test_df_list = [] 
trace_id_list = []
with tqdm(desc="processing traces", total=len(test_traces)) as progress_bar:
    for trace in test_traces:
        train_df, test_df = create_train_test_trace(complete_df, trace)
        progress_bar.update(1)
        if len(test_df) > 0:
            trace_train_df_list.append(train_df)
            trace_test_df_list.append(test_df)
            trace_id_list.append(trace)

processing traces: 100%|█████████████████████████████████████████████████████████| 4234/4234 [12:31:16<00:00, 10.65s/it]


In [18]:
# @contextlib.contextmanager
# def tqdm_joblib(tqdm_object):
#     """Context manager to patch joblib to report into tqdm progress bar given as argument"""
#     class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
#         def __call__(self, *args, **kwargs):
#             tqdm_object.update(n=self.batch_size)
#             return super().__call__(*args, **kwargs)

#     old_batch_callback = joblib.parallel.BatchCompletionCallBack
#     joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
#     try:
#         yield tqdm_object
#     finally:
#         joblib.parallel.BatchCompletionCallBack = old_batch_callback
#         tqdm_object.close()

In [19]:
# from joblib import Parallel, delayed

# def process_trace(trace):
#     train_df, test_df = create_train_test_trace(complete_df, trace)
#     if len(test_df) > 0:
#         return (train_df, test_df, trace)
#     else:
#         return None
# with tqdm_joblib(tqdm(desc="processing traces", total=len(test_traces))) as progress_bar:
#     results = Parallel(n_jobs=-1)(delayed(process_trace)(trace) for trace in test_traces)

# trace_train_df_list = [result[0] for result in results if result is not None]
# trace_test_df_list = [result[1] for result in results if result is not None]
# trace_id_list = [result[2] for result in results if result is not None]

In [20]:
def create_triplets(df):
    triplets = []
    for i in range(len(df)):
        head = df.iloc[i]['dm']
        tail = df.iloc[i]['um']
        rel = df.iloc[i]['rpctype']
        triplets.append([head,rel,tail])
    return triplets

In [21]:
def create_tsv(triplets, file_name):
    with open(file_name, 'w') as file:
        for elem in triplets: 
            file.write(f'{elem[0]}	{elem[1]}	{elem[2]}\n')
        file.close()

In [22]:

# Create a directory
directory = "MSCallGraph_traces"
if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"{directory} created successfully")
else:
    print(f"{directory} already exists")

    # Create a directory
directory = "MSCallGraph_traces/train"
if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"{directory} created successfully")
else:
    print(f"{directory} already exists")
    # Create a directory
directory = "MSCallGraph_traces/test"
if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"{directory} created successfully")
else:
    print(f"{directory} already exists")

MSCallGraph_traces created successfully
MSCallGraph_traces/train created successfully
MSCallGraph_traces/test created successfully


In [23]:
with tqdm(desc="writing train tsv", total=len(trace_id_list)) as progress_bar:
    for elem in zip(trace_train_df_list,trace_id_list):
        create_tsv(create_triplets(elem[0]),f"MSCallGraph_traces/train/{elem[1]}_transductive_train_io.tsv")
        progress_bar.update(1)

writing train tsv: 100%|███████████████████████████████████████████████████████████| 2200/2200 [00:20<00:00, 105.12it/s]


In [24]:
with tqdm(desc="writing test tsv", total=len(trace_id_list)) as progress_bar:  
    for elem in zip(trace_test_df_list,trace_id_list):
        create_tsv(create_triplets(elem[0]),f"MSCallGraph_traces/test/{elem[1]}_transductive_test_io.tsv")
        progress_bar.update(1)

writing test tsv: 100%|████████████████████████████████████████████████████████████| 2200/2200 [00:12<00:00, 171.20it/s]


In [2]:
!mv MSCallGraph_traces MSCallGraph_traces_io

In [3]:

def zip_folder(folder_path, output_path):
    """
    Compresses a folder into a zip file.

    :param folder_path: The path to the folder to be compressed.
    :param output_path: The path and filename of the resulting zip file.
    """

    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        # Iterate over all the files in the folder and add them to the zip file.
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zip_file.write(os.path.join(root, file))

    print(f"{output_path} created successfully!")


# Example usage
folder_path = "MSCallGraph_traces_io"
output_path = folder_path+".zip"

print("Zipping")
zip_folder(folder_path, output_path)
print("Done")

Zipping
MSCallGraph_traces_io.zip created successfully!
Done
